In [1]:
import os
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import pandas as pd

In [2]:
path = os.getcwd() # donde esta el jupyter 
#folder= path + "/dataset/series_mapa/layer04/" # donde estan los .nc

folder= path + "/dataset/esc307-9.2Mw/level0/"
print(folder)

/home/jorge/tsunami-warning/dataset/esc307-9.2Mw/level0/


In [3]:
files = []
for file in os.listdir(folder):
    if file.endswith("01.nc"):
        files.append(file)
print(files)

['level01.nc']


In [4]:
files=['layer04E390.nc'] #pueba files original es el de arriba

**comentario:** agrega todos los archivos .nc a lista pata hacer analisis de cada uno.

In [5]:
for file in files:
    print(file)
    rutafile = folder  + file
    
    f=Dataset(rutafile,'r') # abrir file
    print(f.variables)
    # variables a utilizar
    lon=f.variables['lon'][:]-360 # corregir longitud
    lat=f.variables['lat'][:]
    eta = f.variables['eta'][0]
    nx=eta.shape[1]
    ny=eta.shape[0]
    bat=f.variables['deformed_bathy'][:]
    time = -1
    for e in range(len(f.variables['eta'])):
        time+=1
        fig, ax = plt.subplots(figsize=(10,8),dpi=100)
        eta = f.variables['eta'][e]-f.variables['eta'][0] 
        filtro = eta + f.variables['eta'][0] # lugar que no hay agua
        eta[np.where(eta==0.0)] = np.NaN # altura de agua, falta restarle la topografi
        bat[np.where(bat>0.0)] = np.NaN # topografia

        minz=-5
        maxz=20
        curvas=100
        delta = (maxz-minz)/curvas
        contour=np.linspace(minz,maxz,curvas)

        cs = ax.contourf(lon,lat,eta,curvas,alpha=.9,cmap=cm.get_cmap('jet'),vmin=-15,vmax=15)
        plt.gca().set_aspect('equal', adjustable='box')
        plt.title('Mapa inundación ' + file[-7:-5] +' '+ file[-5]+'.'+file[-4]+' Mw, time: '+ str(time*10)+' min.')
        plt.xlabel('LONGITUDE')
        plt.ylabel('LATITUDE')
        divider = make_axes_locatable(plt.gca())
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cb=fig.colorbar(cs,format='%.2f',cax=cax)
        cb.set_label('METERS')

        if not os.path.exists(rutafile.split('.nc')[0]): # crea folder si no existe
            os.makedirs(rutafile.split('.nc')[0])

        fig.savefig(rutafile.split('.nc')[0]+'/'+file.split('.nc')[0]+'_time'+str(e+10)+'.png',
                       dpi=300,bbox_inches = 'tight',pad_inches = 0)

        #plt.close() no mostrar imagenes si son muchos archivos 
plt.show()

level01.nc
OrderedDict([('lon', <class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    long_name: longitude
    units: degrees
unlimited dimensions: 
current shape = (391,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('lat', <class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    long_name: latitude
    units: degrees
unlimited dimensions: 
current shape = (1201,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('grid_lon', <class 'netCDF4._netCDF4.Variable'>
float64 grid_lon(grid_lon)
    long_name: Grid longitude
    units: degrees
unlimited dimensions: 
current shape = (391,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('grid_lat', <class 'netCDF4._netCDF4.Variable'>
float64 grid_lat(grid_lat)
    long_name: Grid latitude
    units: degrees
unlimited dimensions: 
current shape = (1201,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('original_bathy', <class 'netCDF4._netCDF4.Variable'>
float32 original_bath

KeyError: 'eta'

In [7]:
for file in files:
    print(file)
    rutafile = folder  + file
    
    f=Dataset(rutafile,'r') # abrir file
    print(f.keys())

level01.nc


AttributeError: NetCDF: Attribute not found

In [8]:
15*4.5

67.5